In [ ]:
import os, sys
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Merge, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam, Adamax
from keras.models import model_from_yaml
from keras.regularizers import l1, l2
from load_dataset import get_data, dataset
from utils import print_metadata, get_dgaze_frames_count, split_data, plot_gaze_points, save_model, load_model

from sklearn import preprocessing
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd
import copy
import cv2 

from numpy.random import seed
seed(33)

from tensorflow import set_random_seed
set_random_seed(26)

import random 
random.seed(10)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# from keras import backend as k
# import tensorflow as tf
# config = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1,
# allow_soft_placement=True, device_count = {'CPU': 1})
# sess = tf.Session(graph=tf.get_default_graph(),config=config)
# k.set_session(sess)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
data_path = '/ssd_scratch/cvit/isha/eye_gaze_mapping/DGM_final2/dataset_samples_callibrated/'
model_save = '/ssd_scratch/cvit/isha/DGAZE2/DGAZE_weights/weights_proposed_approach_lefteye_righteye_FF'
drivers = os.listdir(data_path)
ndrivers = len(drivers)
sequences = 112

# Driver_data is dict contatining drivers user1, user 2.....etc. For each driver, we have 112 sequences and for   
# each sequence we have features like ['face_location', 'headpose_pupil', 'left_eye', 'gaze_point', 'right_eye'] 
driver_data = get_data(data_path, drivers, sequences)

# Print the total numer of frames in the dataset
get_dgaze_frames_count(driver_data, drivers)

# Prints the DGAZE Metadata including list of drivers, sequences and features 
#print_metadata(driver_data, ['drivers', 'sequences', 'features'])
print_metadata(driver_data, ['features'])

In [ ]:
seq_range = np.arange(10, sequences+1)
nsequences = len(seq_range)
ndrivers = len(drivers)
random.shuffle(drivers)

dsplit = [int(0.8*ndrivers),int(0.1*ndrivers), int(0.1*ndrivers)]
gp_split = [int(0.6*nsequences),int(0.2*nsequences), int(0.2*nsequences)]
data_split = split_data(drivers, seq_range, dsplit, gp_split)

In [ ]:
data_split

In [ ]:
# Training dataset
train = dataset(driver_data, data_split['drivers_train'], data_split['sequence_train'])

# Validation dataset
val = dataset(driver_data, data_split['drivers_val'], data_split['sequence_val'])

# Test dataset
test = dataset(driver_data, data_split['drivers_test'], data_split['sequence_test'])

In [ ]:
print(train['left_eye'].shape, train['right_eye'].shape, train['headpose_pupil'].shape, \
      train['face_location'].shape, train['face_features'].shape, train['gaze_point'].shape)

print(val['left_eye'].shape, val['right_eye'].shape, val['headpose_pupil'].shape, \
      val['face_location'].shape, val['face_features'].shape, val['gaze_point'].shape)

print(test['left_eye'].shape, test['right_eye'].shape, test['headpose_pupil'].shape, \
      test['face_location'].shape, test['face_features'].shape, test['gaze_point'].shape)

print("Total number of frames -->",train['gaze_point'].shape[0] + val['gaze_point'].shape[0]\
      + test['gaze_point'].shape[0])

In [ ]:
plot_gaze_points(data_path, train['gaze_point'])
plot_gaze_points(data_path, val['gaze_point'])
plot_gaze_points(data_path, test['gaze_point'])

In [ ]:
for d in data_split['drivers_val']:
    data_calibrate = dataset(driver_data, [d], np.arange(12,13))
    x = data_calibrate['face_location']
    y = data_calibrate['headpose_pupil']
    print(data_calibrate['face_features'][0])

    cap = cv2.VideoCapture(data_path + d + '/driver_view/sample_10.avi')
    ret, frame = cap.read()
    plt.figure()
    cv2.rectangle(frame, (x[0,2], x[0,0]), (x[0,3], x[0,1]), (255, 255, 255), 6)
    cv2.circle(frame,(int(y[0,6]), int(y[0,7])),3,(255,255,0),40)
    cv2.circle(frame,(int(y[0,4]), int(y[0,5])),3,(255,255,0),40)
    cv2.circle(frame,(int(y[0,9]), int(y[0,10])),3,(255,255,0),40)
    plt.imshow(frame)
    plt.show()

In [ ]:
# scaler = preprocessing.MinMaxScaler()
# train['face_features'] = scaler.fit_transform(train['face_features'])
# val['face_features'] = scaler.transform(val['face_features'])
# test['face_features'] = scaler.transform(test['face_features'])


In [ ]:
model_lefteye = Sequential()
model_lefteye.add(Conv2D(20, kernel_size=(3, 3),activation='relu',input_shape=(36,60,3)))
model_lefteye.add(MaxPooling2D(pool_size=(2, 2)))
model_lefteye.add(Dropout(0.5))
model_lefteye.add(Conv2D(50, (3, 3), activation='relu'))
model_lefteye.add(MaxPooling2D(pool_size=(2, 2)))
model_lefteye.add(Flatten())

model_righteye = Sequential()
model_righteye.add(Conv2D(20, kernel_size=(3, 3),activation='relu',input_shape=(36,60,3)))
model_righteye.add(MaxPooling2D(pool_size=(2, 2)))
model_righteye.add(Dropout(0.5))
model_righteye.add(Conv2D(50, (3, 3), activation='relu'))
model_righteye.add(MaxPooling2D(pool_size=(2, 2)))
model_righteye.add(Flatten())

model_facefeatures = Sequential()
model_facefeatures.add(Dense(16, activation ='relu', input_dim=(10)))

model3 = Sequential()
model3.add(Merge([model_lefteye, model_righteye], mode = 'concat'))
model3.add(Dense(500, activation='relu'))

model = Sequential()
model.add(Merge([model3, model_facefeatures], mode = 'concat'))
model.add(Dense(258, activation='relu'))
#model_merge.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dense(2, activation="linear"))
print(model.summary())

In [ ]:
opt = Adam(lr=0.001, decay=0.1 / 200)
model.compile(loss = 'mae', optimizer = opt )
earlystopping = keras.callbacks.EarlyStopping(monitor = 'val_loss',min_delta = 1, patience =10, verbose =0, mode ='auto')

history = model.fit([train['left_eye'], train['right_eye'], train['face_features'][:,:10]], train['gaze_point'][:,:2], \
                validation_data= ([val['left_eye'], val['right_eye'], val['face_features'][:,:10]],val['gaze_point'][:,:2]),
                epochs = 200, batch_size = 32,  callbacks =[earlystopping], verbose=1, shuffle= True)

save_model(model_save, model)

print(history.history.keys())

In [ ]:
history = model.fit([train['left_eye'], train['right_eye'], train['face_features'][:,:10]], train['gaze_point'][:,:2], \
                validation_data= ([val['left_eye'], val['right_eye'], val['face_features'][:,:10]],val['gaze_point'][:,:2]),
                epochs = 200, batch_size = 32,  callbacks =[earlystopping], verbose=1, shuffle= True)


In [ ]:
plt.plot(history.history['loss'][1:])
plt.plot(history.history['val_loss'][1:])
plt.show()

In [ ]:
model_save

In [ ]:
save_model(model_save, model)

print(history.history.keys())

In [ ]:
def gaze_error(model, data):
    scores = model.evaluate([data['left_eye'], data['right_eye'], data['face_features'][:,:10]], data['gaze_point'][:,:2])
    return scores


In [ ]:
model = load_model(model_save)
model.compile(loss='mae', optimizer=opt)

train_error = gaze_error(model, train)
val_error = gaze_error(model, val)
test_error = gaze_error(model, test)
    
print("Train Error ==> ", train_error)
print("Val Error ==> ",  val_error)
print("Test Error ==> " ,test_error)

In [ ]:
# for driver in data_split['drivers_test']:

#     data_calibrate = dataset(driver_data, [driver], np.arange(1,113))
#     model_merge = load_model(model_save)
    
#     opt = Adam(lr=0.001, decay=0.1 / 200)
#     model_merge.compile(loss='mae', optimizer=opt)
#     scores = model_merge.evaluate([data_calibrate['left_eye'][2000:], data_calibrate['face_features'][2000:]], data_calibrate['gaze_point'][2000:,:2])
#     print("====> Before Calibration", scores)                   

#     history = model_merge.fit([data_calibrate['left_eye'][:2000], data_calibrate['face_features'][:2000]], data_calibrate['gaze_point'][:2000,:2], \
#                     validation_data= ([data_calibrate['left_eye'][2000:], data_calibrate['face_features'][2000:]],data_calibrate['gaze_point'][2000:,:2]),
#                     epochs = 20, batch_size = 32, verbose=1, shuffle= True)

#     scores = model_merge.evaluate([data_calibrate['left_eye'][2000:], data_calibrate['face_features'][2000:]], data_calibrate['gaze_point'][2000:,:2])

#     print("====> After Calibration", scores) 

In [ ]:
te_error = 0; tr_error = 0
for driver in data_split['drivers_val']:
    data_calibrate = dataset(driver_data, [driver], np.arange(1,15))
                         
    te_calibrate = dataset(driver_data, [driver], data_split['sequence_val'])

    print(te_calibrate['gaze_point'].shape)
    opt = Adam(lr=0.0001, decay=0.1 / 200)
    
    model_merge = load_model(model_save)
    for layer in model_merge.layers[:1]:
        layer.trainable = False
        
    model_merge.compile(loss = 'mae', optimizer = opt)
    
    error = gaze_error(model_merge, te_calibrate)
    tr_error += error
    print("Test Error ==> " ,error)
              

        
    model_merge.fit([data_calibrate['left_eye'], data_calibrate['right_eye']], data_calibrate['gaze_point'][:,:2], \
                validation_data= ([te_calibrate['left_eye'], te_calibrate['right_eye']],te_calibrate['gaze_point'][:,:2]),
                epochs = 10, batch_size = 8, callbacks = [earlystopping], verbose=1, shuffle= True)


    error = gaze_error(model_merge, te_calibrate)
    te_error += error
    print("Test Error ==> " ,error)
                         
print("Total test error -->", tr_error/len(data_split['drivers_val']), te_error/len(data_split['drivers_val']))
                         
    

In [ ]:
te_error = 0; tr_error = 0
for driver in data_split['drivers_test']:
    data_calibrate = dataset(driver_data, [driver], np.arange(1,15))
                         
    te_calibrate = dataset(driver_data, [driver], data_split['sequence_test'])

    print(te_calibrate['gaze_point'].shape)
    
    opt = Adam(lr=0.00001, decay=0.1 / 200)
    
    model_merge = load_model(model_save)
    for layer in model_merge.layers[:1]:
        layer.trainable = False
        
    model_merge.compile(loss = 'mae', optimizer = opt)
    
    error = gaze_error(model_merge, te_calibrate)
    tr_error += error
    print("Test Error ==> " ,error)
              

        
    model_merge.fit([data_calibrate['left_eye'], data_calibrate['right_eye']], data_calibrate['gaze_point'][:,:2], \
                validation_data= ([te_calibrate['left_eye'], te_calibrate['right_eye']],te_calibrate['gaze_point'][:,:2]),
                epochs = 10, batch_size = 16, verbose=1, shuffle= True)


    error = gaze_error(model_merge, te_calibrate)
    te_error += error
    print("Test Error ==> " ,error)
                         
print("Total test error -->", tr_error/len(data_split['drivers_test']), te_error/len(data_split['drivers_val']))
                         
    